In [1]:
import numpy as np
import pandas as pd

In [3]:
94133/169

557.0

In [2]:
def estimateparams(X,questions,allbest,allworst,pw,pH,w0=[],tol=.0001,maxiter=100,hessian=0):
    tol=.001
    val=1
    [numofq,nitems]=X.shape
    if len(w0)<nitems or (np.abs(w0)>4).any():
        w=np.zeros(nitems)    
    else:
        w=w0.copy()
    iternum=0
    nb=np.bincount(allbest,minlength=nitems)
    nw=np.bincount(allworst,minlength=nitems)
    try:
        while val>tol and iternum<maxiter:
            zb=np.sum(np.exp(w[questions]),1)
            zw=np.sum(np.exp(-w[questions]),1)
            Pb=((1/(zb)*X.T).T*np.exp(w))#broadcasting
            Pw=((1/(zw)*X.T).T*np.exp(-w))#broadcasting
            gradb=nb-np.sum(Pb,0)
            gradw=-nw+np.sum(Pw,0)
            grad=-(gradb+gradw)+np.dot(w-pw,pH)
            Hb=np.dot(Pb.T,Pb)
            Hbd=-np.sum(Pb*(1-Pb),0)
            Hb+=-np.diag(np.diag(Hb))+np.diag(Hbd)
            Hw=np.dot(Pw.T,Pw)
            Hwd=-np.sum(Pw*(1-Pw),0)
            Hw+=-np.diag(np.diag(Hw))+np.diag(Hwd)
            H=-(Hb+Hw)+pH
            gradl=grad[:-1]
            Hl=H[:-1,:-1]
            #Hlinv=np.linalg.pinv(Hl)
            #Hinvgradl=np.dot(Hlinv,gradl)
            Hinvgradl=np.linalg.solve(Hl,gradl)
            w[:-1]=w[:-1]-Hinvgradl
            val=np.dot(gradl,Hinvgradl)
            iternum+=1
    except np.linalg.linalg.LinAlgError:
        print('error')
        print(numofq)
        if hessian:
            return np.random.randn(nitems)/4,np.eye(nitems-1)
        else:
            return np.random.randn(nitems)/4
    if np.isnan(w).any():
        print('nan')
        print(numofq)
        return np.random.randn(nitems)/4
        #raise Exception('w contains nan')
    if hessian:
        return w,Hl
    else:
        return w

In [33]:
numresp=1634
tasks=15
qpertask=4
itemnum=35
choicedata=np.zeros((numresp*tasks,qpertask+4),dtype=int)
with open('MaxDiff_2.cho') as f:
    for resp in range(numresp):
        respv=np.array(f.readline().strip().split(),dtype=int)
        choicedata[resp*tasks:(resp+1)*tasks,0]=int(respv[0])
        choicedata[resp*tasks:(resp+1)*tasks,1]=range(1,tasks+1)
        for i in range(tasks):
            f.readline()
            for z in range(qpertask):
                vec=np.array(f.readline().strip().split(),dtype=int)
                choicedata[resp*tasks+i,2+z]=np.dot(vec,range(1,itemnum+1))
            best=np.array(f.readline().strip().split(),dtype=int)[0]
            choicedata[resp*tasks+i,-2]=choicedata[resp*tasks+i,best+1]
            f.readline()
            for z in range(qpertask):
                f.readline()
            worst=np.array(f.readline().strip().split(),dtype=int)[0]
            choicedata[resp*tasks+i,-1]=choicedata[resp*tasks+i,worst+1]
choicedata_df=pd.DataFrame(choicedata)
choicedata_df.columns=["RespNum","QNum",
                       "Choice1","Choice2","Choice3","Choice4",
                       "Best","Worst"]
choicedata_df.to_csv("choice_data-MaxDiff_2.csv",index=False)

In [16]:
numresp=557
tasks=12
qpertask=5
itemnum=86
choicedata=np.zeros((numresp*tasks,qpertask+4),dtype=int)
with open('6000.cho') as f:
    for resp in range(numresp):
        respv=np.array(f.readline().strip().split(),dtype=int)
        choicedata[resp*tasks:(resp+1)*tasks,0]=int(respv[0])
        choicedata[resp*tasks:(resp+1)*tasks,1]=range(1,tasks+1)
        for i in range(tasks):
            f.readline()
            for z in range(qpertask):
                vec=np.array(f.readline().strip().split(),dtype=int)
                choicedata[resp*tasks+i,2+z]=np.dot(vec,range(1,itemnum))
            best=np.array(f.readline().strip().split(),dtype=int)[0]
            choicedata[resp*tasks+i,-2]=choicedata[resp*tasks+i,best+1]
            f.readline()
            for z in range(qpertask):
                f.readline()
            worst=np.array(f.readline().strip().split(),dtype=int)[0]
            choicedata[resp*tasks+i,-1]=choicedata[resp*tasks+i,worst+1]
choicedata[choicedata==0]=86

In [17]:
choices=choicedata[:,2:2+qpertask]-1
allbest=choicedata[:,-2]-1
allworst=choicedata[:,-1]-1

In [18]:
[numofq,items]=choices.shape
X=np.zeros((numofq,itemnum))
for z,row in enumerate(choices):
    X[z,row]=1

In [19]:
pw=np.zeros(itemnum)
pH=np.eye(itemnum)*.001

In [20]:
w,H=estimateparams(X,choices,allbest,allworst,pw,pH,w0=[],tol=.0001,maxiter=100,hessian=1)

In [21]:
respparams=np.zeros((numresp,itemnum+2))
for resp in range(numresp):
    respparams[resp,0]=resp
    respparams[resp,1]=resp
    respparams[resp,2:]=estimateparams(X[resp*tasks:(resp+1)*tasks],choices[resp*tasks:(resp+1)*tasks],
                                 allbest[resp*tasks:(resp+1)*tasks],
                                 allworst[resp*tasks:(resp+1)*tasks],w,np.eye(itemnum)*.5)

In [27]:
np.argsort(respparams[:,2:].mean(0))[::-1]

array([55,  7, 79, 84, 83, 74, 77, 67, 68,  1, 71, 21, 63,  6, 62, 25, 73,
       60, 51, 59, 50, 13, 12, 61, 66, 31, 46, 82, 81, 16, 48, 30, 10, 65,
       42, 49, 29, 70,  2, 20, 15, 80, 54, 57, 43, 72, 56, 40, 33,  3,  0,
       75,  4,  8, 69,  5, 52,  9, 34, 27, 58, 11, 76, 47, 64, 19, 23, 18,
       39, 41, 35, 78, 28, 24, 26, 38, 44, 37, 22, 53, 32, 85, 14, 36, 45,
       17])

In [29]:
np.argsort(w)[::-1]

array([55,  7, 79, 84, 83, 74, 77, 67, 68,  1, 71, 21, 63,  6, 62, 25, 73,
       60, 51, 59, 50, 13, 12, 61, 66, 31, 46, 82, 16, 81, 48, 30, 10, 65,
       42, 49, 29, 70,  2, 20, 15, 80, 57, 54, 43, 56, 72, 40, 33,  0,  3,
        4, 75,  8, 69,  5, 52,  9, 34, 27, 58, 11, 76, 47, 64, 19, 23, 18,
       39, 41, 35, 78, 28, 24, 26, 38, 44, 37, 22, 53, 32, 14, 85, 36, 45,
       17])

In [23]:
colname=["Respondent","RLH"]
for i in range(itemnum):
    colname+=["Item_"+str(i+1)]

In [24]:
respparams_df=pd.DataFrame(respparams)
respparams_df.columns=colname
respparams_df.to_csv("ZB_86items.csv",index=False)

In [300]:
def remove(arr):
    newarr=np.copy(arr)
    vals=np.unique(arr)
    for i,val in enumerate(vals):
        newarr[arr==val]=i+1
    return newarr
    

In [275]:
MDdesign=np.zeros((100*15,6),dtype=int)

In [301]:
for i in range(100):
    MDdesign[i*tasks:(i+1)*tasks,0]=i+1
    MDdesign[i*tasks:(i+1)*tasks,1]=range(1,tasks+1)
    MDdesign[i*tasks:(i+1)*tasks,2:]=remove(choices[i*tasks:(i+1)*tasks])

15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15
15


In [278]:
MDdesign_df=pd.DataFrame(MDdesign)

In [280]:
cols=['Version','Set','Item1','Item2','Item3','Item4']

In [281]:
MDdesign_df.columns=cols

In [284]:
MDdesign_df.to_csv("15itemMD_MD15_Design.csv",index=False)

In [ ]:
    for i in range(n):
        betas,allquestions,allbest,allworst,people=simulation(
            numresp,k,samplescheme,batchsize,itemfile,designfile,mis)
        rowstart=i*numresp*nofqsbot
        rowend=(i+1)*numresp*nofqsbot
        choicedata[rowstart:rowend,0]=i+1
        for j,person in enumerate(people):
            choicedata[rowstart+j*nofqsbot:rowstart+(j+1)*nofqsbot,1]=person
            choicedata[rowstart+j*nofqsbot:rowstart+(j+1)*nofqsbot,2]=j
            choicedata[rowstart+j*nofqsbot:rowstart+(j+1)*nofqsbot,3]=[x for x in range(1,nofqsbot+1)]
        choicedata[rowstart:rowend,4:9]=allquestions+1
        choicedata[rowstart:rowend,9]=allbest+1
        choicedata[rowstart:rowend,10]=allworst+1
        allbetas[i*numrows:(i+1)*numrows,0]=i+1
        allbetas[i*numrows:(i+1)*numrows,1]=[x for x in range(20,numrows*20+20,20)]
        allbetas[i*numrows:(i+1)*numrows,2:]=betas
    choicedata_df=pd.DataFrame(choicedata)
    choicedata_df.columns=["RespNum","QNum",
                           "Choice1","Choice2","Choice3","Choice4",
                           "Best","Worst"]
    allbetas_df=pd.DataFrame(allbetas)
    colname=["Iter","RespNum"]
    for i in range(nitems):
        colname+=["Item_"+str(i+1)]
    allbetas_df.columns=colname
    choicedata_df.to_csv("Results/choice_data-"+filename)